### Set up

In [2]:
import json
import os

import azure.identity
import dotenv
import numpy as np
import openai
import pandas as pd

# Set up OpenAI client based on environment variables
dotenv.load_dotenv()
AZURE_OPENAI_SERVICE = os.getenv("AZURE_OPENAI_SERVICE")
AZURE_OPENAI_ADA_DEPLOYMENT = os.getenv("AZURE_OPENAI_ADA_DEPLOYMENT")

azure_credential = azure.identity.DefaultAzureCredential()
token_provider = azure.identity.get_bearer_token_provider(azure_credential,
    "https://cognitiveservices.azure.com/.default")
openai_client = openai.AzureOpenAI(
    api_version="2024-02-15-preview",
    azure_endpoint=f"https://{AZURE_OPENAI_SERVICE}.openai.azure.com",
    azure_ad_token_provider=token_provider)

def get_embedding(text):
    get_embeddings_response = openai_client.embeddings.create(model=AZURE_OPENAI_ADA_DEPLOYMENT, input=text)
    return get_embeddings_response.data[0].embedding
    
def get_embeddings(sentences):
    embeddings_response = openai_client.embeddings.create(model=AZURE_OPENAI_ADA_DEPLOYMENT, input=sentences)
    return [embedding_object.embedding for embedding_object in embeddings_response.data]


### Vector representations

In [6]:
vector = get_embedding("หม้อไฟ") 

In [7]:
print("Vector value:")
display(vector)


Vector value:


[-0.00801532156765461,
 -0.06998042017221451,
 -0.00011859946243930608,
 0.01735127530992031,
 0.08065008372068405,
 0.023431412875652313,
 -0.0540805347263813,
 0.001596036134287715,
 0.003854022826999426,
 0.0052629150450229645,
 0.004161298740655184,
 -0.007008502725511789,
 -0.005942844320088625,
 0.03271506354212761,
 -0.004821614362299442,
 -0.00596572645008564,
 0.005377326160669327,
 -0.012787902727723122,
 -0.03336884081363678,
 -0.011166532523930073,
 -0.039095938205718994,
 -0.008878308348357677,
 0.022999918088316917,
 0.041240330785512924,
 0.017207443714141846,
 -0.0032116856891661882,
 -0.0057140220887959,
 0.022816861048340797,
 0.015219957567751408,
 -0.004184180870652199,
 0.01752125658094883,
 0.018018128350377083,
 0.007806112058460712,
 0.03572244197130203,
 0.013925476931035519,
 0.018894191831350327,
 0.0032214922830462456,
 0.03708230331540108,
 0.026177281513810158,
 0.0063154976814985275,
 -0.010035496205091476,
 -0.06825444847345352,
 -0.011813772842288017,
 

In [8]:
len(vector)

3072

### Document similarity modeled as cosine distance

In [15]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

sentences1 = ['ชาบูชาบู']
sentences2 = ['ชาบูชาบู','ซูชิ','จิ้มจุ่ม','อาหารสุนัข','อเมริกา']

embeddings1 = get_embeddings(sentences1)
embeddings2 = get_embeddings(sentences2)

for i in range(len(sentences1)):
    for j in range(len(sentences2)):
        print(f"{sentences1[i]} \t\t {sentences2[j]} \t\t Score: {cosine_similarity(embeddings1[i], embeddings2[j]):.4f}")

ชาบูชาบู 		 ชาบูชาบู 		 Score: 1.0000
ชาบูชาบู 		 ซูชิ 		 Score: 0.3641
ชาบูชาบู 		 จิ้มจุ่ม 		 Score: 0.3410
ชาบูชาบู 		 อาหารสุนัข 		 Score: 0.2907
ชาบูชาบู 		 อเมริกา 		 Score: 0.1968


In [26]:
import pandas as pd
# FILEPATH: /workspaces/vector-search-demos/vector_embeddings.ipynb
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

sentences1 = ['ชาบูชาบู (Shabu-shabu) - อาหารญี่ปุ่นที่ใช้น้ำซุปร้อนๆ ในการจิ้มเนื้อสัตว์และผักสด มีลักษณะการรับประทานที่ผู้คนรอบโต๊ะสามารถปรุงอาหารได้ด้วยตนเองเหมือนหม้อไฟ.']

sentences2 = [
    'ชาบูชาบู',
    'สุกี้ยากี้ (Sukiyaki) - เป็นอาหารญี่ปุ่นที่มีวิธีการปรุงคล้ายกับชาบูชาบู แต่สุกี้ยากี้จะมีน้ำซุปหวานและเนื้อสัตว์จะถูกจิ้มลงในไข่ดิบก่อนทาน ซึ่งให้ประสบการณ์การรับประทานที่คล้ายคลึงกับหม้อไฟ.',
    'จิ้มจุ่ม - อาหารไทยที่ใช้น้ำซุปร้อนเพื่อต้มเนื้อสัตว์และผัก ให้ประสบการณ์การรับประทานที่คล้ายกับหม้อไฟ โดยมีการใช้น้ำจิ้มรสเข้มข้นเพื่อเพิ่มรสชาติ.'
]

embeddings1 = get_embeddings(sentences1)
embeddings2 = get_embeddings(sentences2)

scores = []
for i in range(len(sentences1)):
    for j in range(len(sentences2)):
        score = cosine_similarity(embeddings1[i], embeddings2[j])
        scores.append([sentences1[i], sentences2[j], score])

df = pd.DataFrame(scores, columns=['Sentence 1', 'Sentence 2', 'Score'])
print(df)


                                          Sentence 1  \
0  ชาบูชาบู (Shabu-shabu) - อาหารญี่ปุ่นที่ใช้น้ำ...   
1  ชาบูชาบู (Shabu-shabu) - อาหารญี่ปุ่นที่ใช้น้ำ...   
2  ชาบูชาบู (Shabu-shabu) - อาหารญี่ปุ่นที่ใช้น้ำ...   

                                          Sentence 2     Score  
0                                           ชาบูชาบู  0.595230  
1  สุกี้ยากี้ (Sukiyaki) - เป็นอาหารญี่ปุ่นที่มีว...  0.546309  
2  จิ้มจุ่ม - อาหารไทยที่ใช้น้ำซุปร้อนเพื่อต้มเนื...  0.540124  


### Vector search

In [30]:
# Compute vector for query
query = "ชาบูชาบู (Shabu-shabu) - อาหารญี่ปุ่นที่ใช้น้ำซุปร้อนๆ ในการจิ้มเนื้อสัตว์และผักสด มีลักษณะการรับประทานที่ผู้คนรอบโต๊ะสามารถปรุงอาหารได้ด้วยตนเองเหมือนหม้อไฟ."

embeddings_response = openai_client.embeddings.create(model=AZURE_OPENAI_ADA_DEPLOYMENT, input=[query])
vector = embeddings_response.data[0].embedding

# Compute embeddings for food names
food_names = ['ชาบูชาบู', 
              'สุกี้ยากี้ (Sukiyaki) - เป็นอาหารญี่ปุ่นที่มีวิธีการปรุงคล้ายกับชาบูชาบู แต่สุกี้ยากี้จะมีน้ำซุปหวานและเนื้อสัตว์จะถูกจิ้มลงในไข่ดิบก่อนทาน ซึ่งให้ประสบการณ์การรับประทานที่คล้ายคลึงกับหม้อไฟ', 
              'จิ้มจุ่ม - อาหารไทยที่ใช้น้ำซุปร้อนเพื่อต้มเนื้อสัตว์และผัก ให้ประสบการณ์การรับประทานที่คล้ายกับหม้อไฟ โดยมีการใช้น้ำจิ้มรสเข้มข้นเพื่อเพิ่มรสชาติ.', 
              'อาหารสุนัข', 
              'อเมริกา']
food_vectors = get_embeddings(food_names)

# Compute cosine similarity between query and each food name
scores = []
for i in range(len(food_names)):
    scores.append((food_names[i], cosine_similarity(vector, food_vectors[i])))

# Display the top 10 results
df = pd.DataFrame(scores, columns=['Food Name', 'Score'])
df = df.sort_values('Score', ascending=False)
df.head(10)


,Food Name,Score
0,ชาบูชาบู,0.595230
1,สุกี้ยากี้ (Sukiyaki) - เป็นอาหารญี่ปุ่นที่มีว...,0.569158
2,จิ้มจุ่ม - อาหารไทยที่ใช้น้ำซุปร้อนเพื่อต้มเนื...,0.540145
3,อาหารสุนัข,0.256622
4,อเมริกา,0.113191
